# Exploratory Data Analysis with SparkSQL and SparkR

[**Introduction to Apache Spark with R by J. A. Dianes**](https://github.com/jadianes/spark-r-notebooks)

In this notebook we will introduce more advanced concepts about SparkSQL with R that you can find in the [SparkR documentation](http://spark.apache.org/docs/latest/sparkr.html), applied to the [2013 American Community Survey](http://www.census.gov/programs-surveys/acs/data/summary-file.html) housing data. These concepts are related with data frame manipulation, including data slicing, summary statistics, and aggregations. We will use them in combination with [ggplot2](http://ggplot2.org/) visualisations in order to perform [Exploratory Data Analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis).   

## Creating a SparkSQL context and loading data

In order to explore our data, we first need to load it into a SparkSQL data frame. But first we need to init a SparkSQL context. The first thing we need to do is to set up some environment variables and library paths as follows. Remember to replace the value assigned to `SPARK_HOME` with your Spark home folder.  

In [2]:
# Set Spark home and R libs
Sys.setenv(SPARK_HOME='/home/cluster/spark-1.5.0-bin-hadoop2.6')
.libPaths(c(file.path(Sys.getenv('SPARK_HOME'), 'R', 'lib'), .libPaths()))

Now we can load the `SparkR` library as follows.

In [3]:
library(SparkR)


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    filter, na.omit

The following objects are masked from ‘package:base’:

    intersect, rbind, sample, subset, summary, table, transform



And now we can initialise the Spark context as [in the official documentation](http://spark.apache.org/docs/latest/sparkr.html#starting-up-sparkcontext-sqlcontext). In our case we are use a standalone Spark cluster with one master and seven workers. If you are running Spark in local node, use just `master='local'`. Additionally, we require a Spark package from Databricks to read CSV files (more on this in the [previous notebook](https://github.com/jadianes/spark-r-notebooks/blob/master/notebooks/nb1-spark-sql-basics/nb1-spark-sql-basics.ipynb)). 

In [4]:
sc <- sparkR.init(master='spark://169.254.206.2:7077', sparkPackages="com.databricks:spark-csv_2.11:1.2.0")

Launching java with spark-submit command /home/cluster/spark-1.5.0-bin-hadoop2.6/bin/spark-submit  --packages com.databricks:spark-csv_2.11:1.2.0 sparkr-shell /tmp/RtmpN82CWe/backend_port3be62488fbc0 


And finally we can start the SparkSQL context as follows.

In [5]:
sqlContext <- sparkRSQL.init(sc)

Now that we have our SparkSQL context ready, we can use it to load our CSV data into data frames. We have downloaded our [2013 American Community Survey dataset](http://www.census.gov/programs-surveys/acs/data/summary-file.html) files in [notebook 0](https://github.com/jadianes/spark-r-notebooks/tree/master/notebooks/nb0-starting-up/nb0-starting-up.ipynb), so they should be stored locally. Remember to set the right path for your data files in the first line, ours is `/nfs/data/2013-acs/ss13husa.csv`.  

In [8]:
housing_a_file_path <- file.path('', 'nfs','data','2013-acs','ss13husa.csv')
housing_b_file_path <- file.path('', 'nfs','data','2013-acs','ss13husb.csv')

Now let's read into a SparkSQL dataframe. We need to pass four parameters in addition to the `sqlContext`:  

- The file path.  
- `header='true'` since our `csv` files have a header with the column names. 
- Indicate that we want the library to infer the schema.  
- And the source type (the Databricks package in this case). 

And we have two separate files for both, housing and population data. We need to join them.

In [9]:
housing_a_df <- read.df(sqlContext, 
                        housing_a_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')

In [10]:
housing_b_df <- read.df(sqlContext, 
                        housing_b_file_path, 
                        header='true', 
                        source = "com.databricks.spark.csv", 
                        inferSchema='true')

In [11]:
housing_df <- rbind(housing_a_df, housing_b_df)

Let's check that we have everything there by counting the files and listing a few of them.

In [12]:
nrow(housing_df)

[1] 1476313

In [13]:
head(housing_df)

,RT,SERIALNO,DIVISION,PUMA,REGION,ST,ADJHSG,ADJINC,WGTP,NP,ellip.h,wgtp71,wgtp72,wgtp73,wgtp74,wgtp75,wgtp76,wgtp77,wgtp78,wgtp79,wgtp80
1,H,84,6,2600,3,1,1000000,1007549,0,1,⋯,0,0,0,0,0,0,0,0,0,0
2,H,154,6,2500,3,1,1000000,1007549,51,4,⋯,86,53,59,84,49,15,15,20,50,16
3,H,156,6,1700,3,1,1000000,1007549,449,1,⋯,161,530,601,579,341,378,387,421,621,486
4,H,160,6,2200,3,1,1000000,1007549,16,3,⋯,31,24,33,7,7,13,18,23,23,5
5,H,231,6,2400,3,1,1000000,1007549,52,1,⋯,21,18,37,49,103,38,49,51,46,47
6,H,286,6,900,3,1,1000000,1007549,76,1,⋯,128,25,68,66,80,26,66,164,88,24


## Data selection

- select
- filter and contains
- between
- isNull, isNotNull
- startsWith

## Summary statistics

- describe (with column selection)
- mean
- min
- max 
- etc

## Data aggregation and sorting

- withColumn
- aggregate
- orderBy